# Project 1: SAT & ACT Analysis
### Data Cleaning and Feature Engineering

## Purpose of this Notebook

_Standardized college tests are the key for high school seniors need to have in order to access the world of college. Evaluating students across different locations, demographics, and societies through a standardized test is one of the ways that colleges can get an objective idea of how prepared a student is to succeed in college._ 

_There are two main standardized college entrance exams: the **SAT** (Scholastic Aptitude Test) and the **ACT** (American College Testing). We will be comparing various facets of both tests to discover any interesting patterns, as well as focus in on **expanding the SAT participation rate in Alaska.**_

## Executive Summary
### Contents:
- [Data Importing and Cleaning](#Importing-and-Cleaning-Data)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
- [Data Visualization](#Visualize-the-data)
- [Descriptive and Inferential Statistics](#Descriptive-and-Inferential-Statistics)
- [Outside Research](#Outside-Research)
- [Conclusions and Recommendations](#Conclusions-and-Recommendations)

In [1]:
# Library and function imports:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Separate Python file with simple functions to clean and engineer the data
from simple_functions import standardizer, pct_changer, remove_pct, remove_commas
from simple_functions import remove_periods, divide_million, stan_dev, draw_heatmap

## Importing and Cleaning Data

In [2]:
# Loaded the 2017 ACT and SAT data in Pandas DataFrames
# This data was manually crafted from information on the SAT and ACT website.
sat17 = pd.read_csv('./data/sat_2017.csv');
act17 = pd.read_csv('./data/act_2017.csv');

In [3]:
# Taking a preliminary look at the data
sat17.head()

,State,Participation,Evidence-Based Reading and Writing,Math,Total
0,Alabama,5%,593,572,1165
1,Alaska,38%,547,533,1080
2,Arizona,30%,563,553,1116
3,Arkansas,3%,614,594,1208
4,California,53%,531,524,1055


In [4]:
act17.head()

,State,Participation,English,Math,Reading,Science,Composite
0,Alabama,100%,18.9,18.4,19.7,19.4,19.2
1,Alaska,65%,18.7,19.8,20.4,19.9,19.8
2,Arizona,62%,18.6,19.8,20.1,19.8,19.7
3,Arkansas,100%,18.9,19.0,19.7,19.5,19.4
4,California,31%,22.5,22.7,23.1,22.2,22.8


### Initial Impressions of the Data
*Both sets of data report the subscore and total score averages for every state for their respective test. Included in both data sets is participation rate per state. ACT has a score range from 1 to 36 and the SAT has a score range from 400 to 1600. All data is within appropriate bounds (but closer inspection on the dataset reveals typo errors with Maryland's 2017 SAT Math score and 2017 ACT Science score).*

In [5]:
# Used .loc() to correct Maryland's SAT Math value
sat17.loc[20,'Math'] = 524

# Used .loc() to correct Maryland's ACT Science value
act17.loc[20, 'Science'] = 23.2

In [6]:
# Taking a closer look at the data types, to see if they correctly describe the data
# Any discrepancies can mean errors in the data
sat17.dtypes

State                                 object
Participation                         object
Evidence-Based Reading and Writing     int64
Math                                   int64
Total                                  int64
dtype: object

In [7]:
act17.dtypes

State             object
Participation     object
English          float64
Math             float64
Reading          float64
Science          float64
Composite         object
dtype: object

*For analysis, ACT Composite scores should be a float, but closer inspection reveals that there is an error in Wyoming's Composite score. Furthermore, participation (both for the ACT and SAT) would be most useful as a float value for the purposes of analysis and should, therefore, be converted.*

In [8]:
# Correcting the participation data
# Function found in the separate Python file

sat17['Participation'] = sat17['Participation'].map(remove_pct)
act17['Participation'] = act17['Participation'].map(remove_pct)

In [9]:
# Correcting Wyoming's ACT Composite score
act17.loc[50, 'Composite'] = 20.2

In [10]:
# Now that Wyoming's ACT Composite score is a float, 
# We can change the ACT Composite score type from object to float
act17['Composite'] = act17['Composite'].astype('float64')

In [11]:
# Confirming corrected dtypes
act17.dtypes

State             object
Participation      int64
English          float64
Math             float64
Reading          float64
Science          float64
Composite        float64
dtype: object

In [12]:
# Changing column names since we'll be combining 2018 test data
# Renaming using a dictionary
act_rename = {
    'State' : 'state',
    'Composite' : '2017_act_total',
    'English' : '2017_act_eng',
    'Math' : '2017_act_math',
    'Reading' : '2017_act_read',
    'Science' : '2017_act_science',
    'Participation' : '2017_act_part'
}

sat_rename = {
    'State': 'state',
    'Participation' : '2017_sat_part',
    'Evidence-Based Reading and Writing' : '2017_sat_read',
    'Math' : '2017_sat_math',
    'Total' : '2017_sat_total'   
}

act17.rename(act_rename, axis=1, inplace=True)
sat17.rename(sat_rename,axis=1,inplace=True)

In [13]:
# At this point, we need to merge the 2017 SAT and ACT data into one dataframe
# Used the pd.merge() function to merge DataFrames
merge = pd.merge(sat17,act17,on='state')

In [14]:
# Check to confirm changes were made correctly
merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 11 columns):
state               51 non-null object
2017_sat_part       51 non-null int64
2017_sat_read       51 non-null int64
2017_sat_math       51 non-null int64
2017_sat_total      51 non-null int64
2017_act_part       51 non-null int64
2017_act_eng        51 non-null float64
2017_act_math       51 non-null float64
2017_act_read       51 non-null float64
2017_act_science    51 non-null float64
2017_act_total      51 non-null float64
dtypes: float64(5), int64(5), object(1)
memory usage: 4.8+ KB


In [15]:
# Saving the 2017 SAT and ACT merged data to an external .csv file
merge.to_csv('./data/combined_2017.csv')

## 2018 Data Import and Cleaning

In [16]:
# We also have available the 2018 SAT and ACT data files
# Reading in the appropriate data sets into DataFrames
act18 = pd.read_csv('./data/act_2018.csv')
sat18 = pd.read_csv('./data/sat_2018.csv')

In [17]:
# Changing participation rates into correct format, renaming columns,
# dropping 'National' entry in ACT, merging the 2018 data, 
# and saving the resulting .csv file
sat18['Participation'] = sat18['Participation'].map(remove_pct)
act18['Participation'] = act18['Participation'].map(remove_pct)

In [18]:
sat18.rename({'State' : 'state', 
              'Total' : '2018_sat_total', 
              'Participation':'2018_sat_part',
              'Evidence-Based Reading and Writing' : '2018_sat_read',
              'Math' : '2018_sat_math'}, axis=1, inplace=True)
act18.rename({'State': 'state', 
              'Composite': '2018_act_total',
              'Participation' : '2018_act_part',},axis=1, inplace=True)

In [19]:
# There was an extraneous row called "National" in the 2018 ACT data
# Dropped the row, as it was unncessary for analysis
act18[act18['state']=='National']
act18 = act18.drop(27)

_At this point, I had a difficult time getting the resulting merged file to have the correct number of entries (51). I looked into the original files and realized that 'Oklahoma' and 'Mississippi' were misspelled in the SAT 2018 file. I went ahead and manually changed those values, although doing so in Pandas would not have been difficult._

In [20]:
# Merging the 2018 SAT and ACT data into one dataframe
merge18=pd.merge(sat18,act18,on='state')
merge18.to_csv('./data/combined_2018.csv')

In [21]:
# Confirming if merge was successful
merge18.shape

(51, 7)

In [22]:
# Merging the 2017 and 2018 data
final = pd.merge(merge,merge18,on='state')

In [23]:
# Checking out the data to see if the process was successful
final.shape

(51, 17)

_I wanted to standardize the ACT and SAT scores so that I can compare them directly later in the project. I decided to only standardize the composite (total) scores for both ACT and SAT. I created the function in the separate Python file._

In [24]:
final['2017_sat_total_standardized'] = standardizer(final['2017_sat_total'])
final['2018_sat_total_standardized'] = standardizer(final['2018_sat_total'])
final['2017_act_total_standardized'] = standardizer(final['2017_act_total'])
final['2018_act_total_standardized'] = standardizer(final['2018_act_total'])

In [25]:
final.head()

,state,2017_sat_part,2017_sat_read,2017_sat_math,2017_sat_total,2017_act_part,2017_act_eng,2017_act_math,2017_act_read,2017_act_science,...,2018_sat_part,2018_sat_read,2018_sat_math,2018_sat_total,2018_act_part,2018_act_total,2017_sat_total_standardized,2018_sat_total_standardized,2017_act_total_standardized,2018_act_total_standardized
0,Alabama,5,593,572,1165,100,18.9,18.4,19.7,19.4,...,6,595,571,1166,100,19.1,0.424770,0.493207,-1.159348,-1.143654
1,Alaska,38,547,533,1080,65,18.7,19.8,20.4,19.9,...,43,562,544,1106,33,20.8,-0.503344,-0.150381,-0.859466,-0.332903
2,Arizona,30,563,553,1116,62,18.6,19.8,20.1,19.8,...,29,577,572,1149,66,19.2,-0.110260,0.310857,-0.909446,-1.095963
3,Arkansas,3,614,594,1208,100,18.9,19.0,19.7,19.5,...,5,592,576,1169,100,19.4,0.894287,0.525386,-1.059388,-1.000580
4,California,53,531,524,1055,31,22.5,22.7,23.1,22.2,...,60,540,536,1076,27,22.7,-0.776319,-0.472174,0.639945,0.573230


_Next, I also wanted to calculate the percent change between 2017 and 2018 scores and participation rates. I created a function for this purpose in the separate Python file._

In [26]:
final['sat_score_pct_change'] = pct_changer(final['2017_sat_total'], final['2018_sat_total'])
final['sat_partication_pct_change'] = pct_changer(final['2017_sat_part'], final['2018_sat_part'])
final['act_score_pct_change'] = pct_changer(final['2017_act_total'], final['2018_act_total'])
final['act_participation_pct_change'] = pct_changer(final['2017_act_part'], final['2018_act_part'])

In [27]:
final.head()

,state,2017_sat_part,2017_sat_read,2017_sat_math,2017_sat_total,2017_act_part,2017_act_eng,2017_act_math,2017_act_read,2017_act_science,...,2018_act_part,2018_act_total,2017_sat_total_standardized,2018_sat_total_standardized,2017_act_total_standardized,2018_act_total_standardized,sat_score_pct_change,sat_partication_pct_change,act_score_pct_change,act_participation_pct_change
0,Alabama,5,593,572,1165,100,18.9,18.4,19.7,19.4,...,100,19.1,0.424770,0.493207,-1.159348,-1.143654,0.000858,0.200000,-0.005208,0.000000
1,Alaska,38,547,533,1080,65,18.7,19.8,20.4,19.9,...,33,20.8,-0.503344,-0.150381,-0.859466,-0.332903,0.024074,0.131579,0.050505,-0.492308
2,Arizona,30,563,553,1116,62,18.6,19.8,20.1,19.8,...,66,19.2,-0.110260,0.310857,-0.909446,-1.095963,0.029570,-0.033333,-0.025381,0.064516
3,Arkansas,3,614,594,1208,100,18.9,19.0,19.7,19.5,...,100,19.4,0.894287,0.525386,-1.059388,-1.000580,-0.032285,0.666667,0.000000,0.000000
4,California,53,531,524,1055,31,22.5,22.7,23.1,22.2,...,27,22.7,-0.776319,-0.472174,0.639945,0.573230,0.019905,0.132075,-0.004386,-0.129032


_I decided to also add four separate datasets to the final dataframe,_ [2016 median income](https://www.census.gov/content/dam/Census/library/publications/2017/acs/acsbr16-02.pdf), [2014 college bound high schoolers](http://www.higheredinfo.org/dbrowser/index.php?submeasure=63&year=2014&level=nation&mode=data&state=0), [2014-2015 Education Revenue by State](https://nces.ed.gov/programs/digest/d17/tables/dt17_235.20.asp?current=yes) and [2017 State Population Estimates](https://www.census.gov/data/tables/2017/demo/popest/state-total.html#par_textimage_1574439295). _The sources are linked. Although the years are not exactly matched up to the 2017 and 2018 tests, this is the most recent information available, and I believe will add valuable insight to the discussion. (Note: District of Columbia information was unavailable for 2014 college bound seniors)_

In [28]:
# Adding four additional datasets to final,
# 2016 median income by state, 2014 college bound high schoolers,
# 2014-2015 education revenue by state, and 2017 population estimates by state.
median_income = pd.read_csv('./data/median_income_2016.csv')
college_bound = pd.read_csv('./data/college_bound.csv')
revenue = pd.read_csv('./data/education_revenue.csv')
pop = pd.read_csv('./data/2017_pop_est.csv')

In [29]:
# The college_bound csv file has a few extraneous columns that I will not be using
college_bound.drop(['high_school_grads','full-time_freshmen_2014'], axis=1, inplace=True)

In [30]:
final = pd.merge(final, median_income, on='state')

In [31]:
final = pd.merge(final, college_bound, on='state')

_Currently, the revenue dataframe holds state education revenues in thousands. To simplify, I converted the numbers into billions by dividing each value by 1,000,000. The functions that I use can be found in the separate Python file._

In [32]:
# The revenue csv file also had 2 NA rows at the bottom of the csv for some reason
# So I dropped them
revenue = revenue.drop(51)
revenue = revenue.drop(52)

In [33]:
# The revenue data had commas and were string objects, therefore had to clean the data
# and divide by one million in order to get the data in billions
revenue['education_revenue'] = revenue['education_revenue'].map(remove_commas)
revenue['education_revenue'] = revenue['education_revenue'].map(divide_million)

In [34]:
final = pd.merge(final,revenue,on='state')

In [35]:
# The data entries for states had an odd period in the beginning of each name
pop['state'] = pop['state'].map(remove_periods)

In [36]:
# Population had commas and were string objects
pop['2017_pop_est'] = pop['2017_pop_est'].map(remove_commas)

In [37]:
final = pd.merge(final, pop, on='state')

In [38]:
# Final check to see if data types were correct
final.dtypes

state                            object
2017_sat_part                     int64
2017_sat_read                     int64
2017_sat_math                     int64
2017_sat_total                    int64
2017_act_part                     int64
2017_act_eng                    float64
2017_act_math                   float64
2017_act_read                   float64
2017_act_science                float64
2017_act_total                  float64
2018_sat_part                     int64
2018_sat_read                     int64
2018_sat_math                     int64
2018_sat_total                    int64
2018_act_part                     int64
2018_act_total                  float64
2017_sat_total_standardized     float64
2018_sat_total_standardized     float64
2017_act_total_standardized     float64
2018_act_total_standardized     float64
sat_score_pct_change            float64
sat_partication_pct_change      float64
act_score_pct_change            float64
act_participation_pct_change    float64


In [39]:
final.head()

,state,2017_sat_part,2017_sat_read,2017_sat_math,2017_sat_total,2017_act_part,2017_act_eng,2017_act_math,2017_act_read,2017_act_science,...,2017_act_total_standardized,2018_act_total_standardized,sat_score_pct_change,sat_partication_pct_change,act_score_pct_change,act_participation_pct_change,median_income,college_bound,education_revenue,2017_pop_est
0,Alabama,5,593,572,1165,100,18.9,18.4,19.7,19.4,...,-1.159348,-1.143654,0.000858,0.200000,-0.005208,0.000000,46257,62.1,7.44,4874747
1,Alaska,38,547,533,1080,65,18.7,19.8,20.4,19.9,...,-0.859466,-0.332903,0.024074,0.131579,0.050505,-0.492308,76440,44.0,2.94,739795
2,Arizona,30,563,553,1116,62,18.6,19.8,20.1,19.8,...,-0.909446,-1.095963,0.029570,-0.033333,-0.025381,0.064516,53558,52.3,9.86,7016270
3,Arkansas,3,614,594,1208,100,18.9,19.0,19.7,19.5,...,-1.059388,-1.000580,-0.032285,0.666667,0.000000,0.000000,44334,63.5,5.28,3004279
4,California,53,531,524,1055,31,22.5,22.7,23.1,22.2,...,0.639945,0.573230,0.019905,0.132075,-0.004386,-0.129032,67739,60.9,74.40,39536653


In [40]:
# Outputting into a final .csv file
final.to_csv('./data/final.csv', index = False)

## Data dictionary of final dataframe to be used for analysis

|**Feature**|**Type**|**Dataset**|**Description**|
|---|---|---|---|
|**state**|_object_|---|The state that the average score and subscore for each test was taken|
|**2017_act_part** | _integer_ | 2017 ACT Data | The participation rate (as a percent) for each state (and District of Columbia) on the ACT in 2017|
|**2017_act_eng** | _float_ | 2017 ACT Data | The average English subscore (out of 36) for each state (and District of Columbia) on the ACT in 2017|
|**2017_act_math** | _float_ | 2017 ACT Data | The average Math subscore (out of 36) for each state (and District of Columbia) on the ACT in 2017|
|**2017_act_read** | _float_ | 2017 ACT Data | The average Reading subscore (out of 36) for each state (and District of Columbia) on the ACT in 2017|
|**2017_act_science** | _float_ | 2017 ACT Data | The average Science subscore (out of 36) for each state (and District of Columbia) on the ACT in 2017|
|**2017_act_total** | _float_ | 2017 ACT Data | The average total score (out of 36) on the ACT test for each state in 2017 |
|**2017_sat_part** | _int_ | 2017 SAT Data | The participation rate (as a percent) for each state (and District of Columbia) on the SAT in 2017|
|**2017_sat_read** | _int_ | 2017 SAT Data | The average Evidence-Based Reading and Writing subscore (out of 800) on the SAT test for each state in 2017|
|**2017_sat_math** | _int_ | 2017 SAT Data | The average Math subscore (out of 800) on the SAT test for each state in 2017|
|**2017_sat_total** | _int_ | 2017 SAT Data | The average total score (out of 1600) on the SAT test for each state in 2017|
|**2018_act_part** | _int_ | 2018 ACT Data | The participation rate (as a percent) for each state (and District of Columbia) on the ACT in 2018|
|**2018_act_total** | _float_ | 2018 ACT Data | The average total score (out of 36) on the ACT test for each state in 2018 |
|**2018_sat_part** | _int_ | 2018 SAT Data | The participation rate (as a percent) for each state (and District of Columbia) on the SAT in 2018|
|**2018_sat_read** | _int_ | 2018 SAT Data | The average Evidence-Based Reading and Writing subscore (out of 800) on the SAT test for each state in 2018|
|**2018_sat_math** | _int_ | 2018 SAT Data | The average Math subscore (out of 800) on the SAT test for each state in 2018|
|**2018_sat_total** | _int_ | 2018 SAT Data | The average total score (out of 1600) on the SAT test for each state in 2018|
|**median_income** | _int_ | 2016 US Census Data | The median income per household for each state in 2016|
|**percent_college_bound** | _float_ | 2014 NCES (National Center for Education Statistics) Data  | The percent of college-bound seniors for each state in 2014|
|**education_revenue** | _int_ | 2014-2015 NCES (National Center for Education Statistics) Data  | The total education revenue (in millions of dollars) for each state in 2014-2015|
|**2017_pop_est** | _int_ | US Census | Estimated population for each state in 2017 by extrapolation from 2010 US Census|
|**2017_sat_total_standardized** | _float_ | 2017 SAT Data | The average total score on the SAT test for each state in 2017 standardized by its mean and standard deviation|
|**2018_sat_total_standardized** | _float_ | 2018 SAT Data | The average total score on the SAT test for each state in 2018 standardized by its mean and standard deviation|
|**2017_act_total_standardized** | _float_ | 2017 ACT Data | The average total score on the ACT test for each state in 2017 standardized by its mean and standard deviation|
|**2018_act_total_standardized** | _float_ | 2018 ACT Data | The average total score on the ACT test for each state in 2018 standardized by its mean and standard deviation|
|**sat_score_pct_change** | _float_ | 2017 and 2018 SAT Data | The percent change between 2017 and 2018 SAT scores|
|**act_score_pct_change** | _float_ | 2017 and 2018 ACT Data | The percent change between 2017 and 2018 ACT scores|
|**sat_participation_pct_change** | _float_ | 2017 and 2018 SAT Data | The percent change between 2017 and 2018 SAT participation|
|**act_participation_pct_change** | _float_ | 2017 and 2018 ACT Data | The percent change between 2017 and 2018 ACT participation|

_This notebook was used for merging and cleaning data, as well as doing some light feature engineering. The next notebook will look at doing exploratory data analysis and creating visualizations._